In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
sample_submit = pd.read_csv('./data/sample_submit.csv',header=None, index_col=0)

In [3]:
train_df = train_df.drop(columns=['id'])
df = train_df.dropna()

X = df.drop(columns=['charges'])
y = df['charges']

X = pd.get_dummies(X)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

# lightGBM

In [13]:
# 勾配ブースティング決定木回帰モデルの学習
from lightgbm import LGBMClassifier
nbr = LGBMClassifier(learning_rate = 0.01,
                    n_estimators = 10000,
                    num_leaves = 4,
                    min_child_samples = 10)
nbr.fit(X_train, y_train,
        eval_set = (X_train, y_train), 
        early_stopping_rounds = 5, 
        verbose = 500)     # verboseは学習の様子をprintするiterationの間隔

Training until validation scores don't improve for 5 rounds
[500]	training's multi_logloss: 0.294834
[1000]	training's multi_logloss: 0.255668
[1500]	training's multi_logloss: 0.225486
[2000]	training's multi_logloss: 0.20291
[2500]	training's multi_logloss: 0.184279
[3000]	training's multi_logloss: 0.168765
[3500]	training's multi_logloss: 0.156349
[4000]	training's multi_logloss: 0.144901
[4500]	training's multi_logloss: 0.134335
[5000]	training's multi_logloss: 0.123189
[5500]	training's multi_logloss: 0.113246
[6000]	training's multi_logloss: 0.105185
[6500]	training's multi_logloss: 0.0988118
[7000]	training's multi_logloss: 0.0920803
[7500]	training's multi_logloss: 0.0863403
[8000]	training's multi_logloss: 0.0805237
[8500]	training's multi_logloss: 0.0747927
[9000]	training's multi_logloss: 0.0695317
[9500]	training's multi_logloss: 0.0652596
[10000]	training's multi_logloss: 0.061441
Did not meet early stopping. Best iteration is:
[10000]	training's multi_logloss: 0.061441


LGBMClassifier(learning_rate=0.01, min_child_samples=10, n_estimators=10000,
               num_leaves=4)

In [14]:
from sklearn.metrics import accuracy_score

# 正答率
acc_train = accuracy_score(y_train, nbr.predict(X_train))
acc_test = accuracy_score(y_test, nbr.predict(X_test))

print("acc_train: "+ str(acc_train) + "   acc_test: "+ str(acc_test))

acc_train: 0.990625   acc_test: 0.859375


In [15]:
from sklearn.metrics import f1_score

f1_train = f1_score(y_train, nbr.predict(X_train), average='macro')
f1_test = f1_score(y_test, nbr.predict(X_test), average='macro')

# 評価結果の表示
print("f1_train:", f1_train, "f1_test", f1_test)

f1_train: 0.9848638658961946 f1_test 0.675422593608471


## 提出

In [16]:
test = test_df.drop(columns=['id'])

test_X = pd.get_dummies(test)
test_X.shape

(400, 11)

In [18]:
sample_submit[1] = nbr.predict(test_X)
sample_submit.to_csv('submit_lgbm.csv', header=None)
sample_submit.head()

,1
0,
13,0
23,2
27,0
28,0
29,0
